In [23]:
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import dask
from dask.distributed import Client
import pathlib

In [24]:
fs = fsspec.filesystem('s3', anon=True)

In [25]:
urls = ['s3://' + f for f in fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")]

In [26]:
client = Client(n_workers=8)

/home/vboxuser/anaconda3/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34285 instead
  warnings.warn(


In [27]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
            outf.write(ujson.dumps(h5chunks.translate()).encode())

In [28]:
pathlib.Path('./jsons/').mkdir(exist_ok=True)

In [29]:
dask.compute(*[dask.delayed(gen_json)(u) for u in urls])

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [38]:
json_list = sorted(glob("./jsons/*.json"))

In [40]:
for js in tqdm(json_list):
    with open(js) as f:
        content = f.read()
        if content:
            m_list.append(fsspec.get_mapper("reference://", fo=ujson.loads(content), remote_protocol="s3", remote_options={"anon": True}))
        else:
            print(f"Warning: Empty file - {js}")

  8%|▊         | 15/194 [00:00<00:01, 114.02it/s]

100%|██████████| 194/194 [00:00<00:00, 528.80it/s]

In [ ]:
ds = xr.open_mfdataset(m_list, combine='nested', concat_dim='t',
                       engine='zarr', coords='minimal', 
                       data_vars='minimal', compat='override',
                       consolidated=False)

In [34]:
json_list = sorted(glob('./jsons/*.json'))

In [35]:
mzz = MultiZarrToZarr(
    json_list,
    remote_protocol="s3",
    remote_options={'anon': True},
    concat_dims='t',
    inline_threshold=0
)


In [36]:
mzz.translate('./combined.json')

JSONDecodeError: Expected object or value

In [37]:
fs = fsspec.filesystem(
    "reference", 
    fo="./combined.json", 
    remote_protocol="s3", 
    remote_options={"anon": True},
    skip_instance_cache=True
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/vboxuser/Desktop/Satip/GOES-16/combined.json/.zmetadata'